# LAB7 Assignment
> The document description are designed by JIa Yanhong in 2022. Oct. 20th
------

# 12012727 王铎磊

## LAB Assignment
### Exercise 1 logistic regression (20 points )
This exercise uses dataset digit01.csv , which has 13 columns, and the last column is the dependent variable. 

This part requires you to implement a `logistic regression` using the pytorch framework (defining a logistic regression class that inherits `nn.module`). To test your model, we provide a dataset `digit01.csv` which is in the **datasets folder**. This dataset requires you to divide the training set and the test set by yourself, and it is recommended that 80% of the training set and 20% of the test set be used.

+ load datasets

In [1]:
########### Write Your Code Here ###########
import pandas as pd
import numpy as np
import torch

df = pd.read_csv("datasets/digit01.csv", header=None).astype("float32")

df.head()
############################################

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0
1,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0
2,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0
3,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0
4,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0


+ Splitting dataset into 80% Training and 20% Testing Data:

In [2]:
########### Write Your Code Here ###########
Y = torch.tensor(np.array(df[12]))
df.drop(12, axis=1, inplace=True)
X = torch.tensor(np.array((df)))
############################################

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.8)

+ Define a LogisticRegression subclass of nn. Module

In [4]:
# Define a LogisticRegression subclass of nn. Module.
########### Write Your Code Here ###########
import torch
from torch import nn

class LR(nn.Module):
    def __init__(self):
        super(LR, self).__init__()
        self.linear = nn.Linear(12, 1)
        self.sigmoid = nn.Sigmoid()
        
        
    def forward(self, x):
        x = self.linear(x)
        x = self.sigmoid(x)
        return x
    

+ Create the model

In [5]:
########### Write Your Code Here ###########
model = LR()
print(model)
############################################

LR(
  (linear): Linear(in_features=12, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)


 + Loss function

In [6]:
########### Write Your Code Here ###########
loss_fn = nn.BCELoss()
############################################

+ The optimizer

In [7]:
########### Write Your Code Here ###########
lr = 0.01
opt = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)
############################################

+ training Model

In [8]:
########### Write Your Code Here ###########
import matplotlib.pyplot as plt

x0 = X[np.where(Y == 0)]
x1 = X[np.where(Y == 1)]


for i in range(500):
    # 前向传播
    y_pred = model(X_train)

    # 计算 loss
    loss = loss_fn(y_pred.squeeze(), y_train)

    # 反向传播
    loss.backward()

    # 更新参数
    opt.step()

    # 清空梯度
    opt.zero_grad()
    if(i % 50 == 0):
        mask = y_pred.ge(0.5).float().squeeze()  #分类
        correct = (mask == y_train).sum() #计算其中的分类正确的个数
        acc = correct.item() / y_train.size(0)#计算准确率
        print("acc=",acc)
############################################

acc= 0.5490196078431373
acc= 0.9607843137254902
acc= 0.9607843137254902
acc= 0.9803921568627451
acc= 0.9803921568627451
acc= 0.9803921568627451
acc= 0.9803921568627451
acc= 0.9803921568627451
acc= 0.9803921568627451
acc= 1.0



+ Model Performance


In [9]:
########### Write Your Code Here ###########
y_predicted_cls = model(X_test).ge(0.5).float().detach().numpy()
############################################

In [10]:
#classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, y_predicted_cls))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00         5
         1.0       1.00      1.00      1.00         8

    accuracy                           1.00        13
   macro avg       1.00      1.00      1.00        13
weighted avg       1.00      1.00      1.00        13



### Exercise 2  Handwriting recognition with MLP

Like last week's lab , your task in this section is also about recognizing handwritten digits, but you are required to use MLP to complete the exercise. It is recommended that you define an MLP class, which is a subclass of `nn.module`.

<font color='red' size=4>Note that your accuracy in this section will directly determine your score.</font>

For this exercise we use the `minist` dataset.

+ load datasets

In [11]:
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor

path = '../Lab7.Neural network/datasets'

train = MNIST(root = path, train = True, transform=ToTensor(), download=True)
test = MNIST(root = path, train = False, transform=ToTensor())

In [12]:
X_train = torch.tensor(train.data, dtype=torch.float32)
y_train = torch.tensor(train.targets, dtype=torch.float32)
X_test = torch.tensor(test.data, dtype=torch.float32)
y_test = torch.tensor(test.targets, dtype=torch.float32)

/var/folders/mh/ypvnyzt13wx0zh1tjkqvy6yh0000gn/T/ipykernel_96867/2333857998.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(train.data, dtype=torch.float32)
/var/folders/mh/ypvnyzt13wx0zh1tjkqvy6yh0000gn/T/ipykernel_96867/2333857998.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.tensor(train.targets, dtype=torch.float32)
/var/folders/mh/ypvnyzt13wx0zh1tjkqvy6yh0000gn/T/ipykernel_96867/2333857998.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(test.data, dtype=torch.

+ Define a MLP subclass of nn. Module

In [13]:
########### Write Your Code Here ###########
import time as t
import numpy as np
from torchmetrics import Accuracy

########### Write Your Code Here ###########
import torch
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor

path = '../Lab7.Neural network/datasets'

train = MNIST(root = path, train = True, transform=ToTensor(), download=True)
test = MNIST(root = path, train = False, transform=ToTensor())

###################

train_iter = torch.utils.data.DataLoader(train, batch_size = 300, shuffle = True, num_workers=2, drop_last=True)
test_iter = torch.utils.data.DataLoader(test, batch_size = 300, shuffle = True, num_workers=2, drop_last=True)

#######################
res = []

from torch import nn

import torch

class ANN(nn.Module):
    def __init__(self):
        super().__init__()
        self.processing = nn.Sequential(
            nn.Linear(28*28, 28*14),
            nn.ReLU(28*14),
            nn.Linear(28*14, 14*14),
            nn.ReLU(14*14),
            nn.Linear(14*14, 7*7)
        )
        
        self.classfier = nn.Sequential(
            nn.Linear(7*7, 16),
            nn.Dropout(p = 0.3),
            nn.BatchNorm1d(16),
            nn.ReLU(inplace=True),
            nn.Dropout(p = 0.5),
            
            nn.Linear(16, 10),
            nn.ReLU(inplace=True)
        )
        
        
    def forward(self, x):
        x = x.reshape(-1, 28*28)
        x = self.processing(x)
        x = self.classfier(x)
        return x

############################################

- Model Training

此段代码谨慎运行，已经将最终模型存储

In [14]:
# ########### Write Your Code Here ###########
# LEARNING_RATE = 0.01
# device = torch.device('mps')
# model = ANN().to(device)
# loss_fn = nn.CrossEntropyLoss().to(device)
# opt = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
# ############################################

# Epochs = 1000
# for i in range(Epochs):
#     last_time = t.time()
#     for cnt, (x, y) in enumerate(train_iter):
#         samples = x.to(device)
#         y = y.to(device)
#         output = model(samples)
        
        
#         loss = loss_fn(output, y.to(device))
#         opt.zero_grad()
        
#         loss.backward()
#         opt.step()
#         y_pred = torch.argmax(output, dim=1).to(device)        
#         acc = (torch.argwhere(y_pred == y)).size(0)/y.size(0)
#         res.append(acc)
        
    
#     mean_acc = np.mean(res)
#     res.clear()
#     this_time = t.time()
#     print("Epoch:{}/{}, step:{}, loss:{:.4f}, time:{:.2f}".format(i + 1, Epochs, i + 1, loss.item(), this_time - last_time))
#     print("acc:{:.4f}".format(mean_acc))
    
    
#     if((i + 1)%(Epochs/(Epochs/10)) == 0):
#         torch.save(model.state_dict(), 'model_final_' + str(model)[:3] + "_" + str(opt)[:7] + "lr=" + str(LEARNING_RATE) + "_" + str(i/(Epochs/100) + 1) + '.pth')
#         print(str(opt)[:7])
#         print("Checkpoint save!")

        

+ Model Test

In [15]:
########### Write Your Code Here ###########
from torchmetrics import Accuracy

model = ANN()
model.load_state_dict(torch.load("Adam_lr_001999.pth", map_location=torch.device("cuda")))

checker = Accuracy()

model.eval()

res = []

for testPoint in range(10):
    acc_list = []
    for cnt, (x, y) in enumerate(test_iter):
        samples = x.to(device)
        output = model(samples)

        y = y.to(device)
        y_pred = torch.argmax(output, dim=1).to(device)        
        acc = (torch.argwhere(y_pred == y)).size(0)/y.size(0)
        acc_list.append(acc)
        
    
    res.append(np.mean(acc_list))


print(np.mean(res))
#0.9793636363636363
############################################

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [17]:
y_predicted_cls = torch.argmax(model(X_test), dim=1)
print(X_test.size())
y_predicted_cls

torch.Size([10000, 28, 28])


tensor([0, 4, 3,  ..., 4, 6, 4])

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_predicted_cls))

### Exercise 3  Questions (10 points )
1.What's the difference between logistic regression and Perceptron?

2.Advantages and disadvantages of neural networks?

3.What is the role of Activation Function in Neural networks?

1. 逻辑回归可以看做最简单的感知器，而感知器可以包括很多其他模型。
2. 处理大样本数据的时候，潜力更大，方便进行复用和转移。训练成本更高，解释性不够强。
3. 用来把数据映射到 (0, 1) 内，并且让靠近边界的样本移动到两端。有利于分类。